In [1]:
import numpy as np
from utils import *
import pandas as pd

In [169]:
data = pd.read_table("data/rte_crowd.txt",header=None)

In [170]:
truth = pd.read_table("data/rte_truth.txt", header=None)


In [171]:
def transform_data(data):
    X = np.array(data)
    rows, _ = X.shape
    n,m,k = np.max(np.array(data), axis=0)
    labels = np.zeros((m,n),dtype=np.int64)-1
    for r in range(rows):
        labels[X[r,1]-1,X[r,0]-1] = X[r,2]-1
    return labels


In [173]:
n,m,k = np.max(np.array(data),axis=0)
print(n,m,k)

800 164 2


In [174]:
labels = transform_data(data)


In [175]:
labels


array([[ 1,  0,  1, ..., -1, -1, -1],
       [ 1,  0,  0, ...,  1,  0,  1],
       [ 1,  0,  1, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]], dtype=int64)

In [ ]:
groups = np.array([i % 3 for i in range(m)])

In [213]:
np.random.seed(0)
groups = np.random.randint(3, size=m)

In [214]:
C_estimated = get_confusion_matrix(k, labels, groups=groups,sym=False)


In [203]:
from sklearn.metrics import confusion_matrix

In [215]:
i = 50
truth_new = truth[truth[0].isin(data[data[1] == i+1][0])]
pred_new = data[data[1] == i+1][data[data[1] == i+1][0].isin(truth_new[0])]


In [216]:
truth_new

,0,1
187,188,1
188,189,2
189,190,1
190,191,2
191,192,2
192,193,2
193,194,1
194,195,1
195,196,2
196,197,2


In [217]:
pred_new

,0,1,2
1876,188,51,1
1886,189,51,2
1896,190,51,1
1906,191,51,2
1916,192,51,2
1926,193,51,2
1936,194,51,1
1946,195,51,1
1956,196,51,1
1966,197,51,1


In [218]:
C = confusion_matrix(pred_new[2],truth_new[1])

In [219]:
np.round(C / np.sum(C, axis=0)[np.newaxis, :],5)


array([[0.88235, 0.3913 ],
       [0.11765, 0.6087 ]])

In [220]:
np.round(C_estimated[i],5)

array([[0.95657, 0.34086],
       [0.04343, 0.65914]])

In [210]:
m, n = labels.shape


In [23]:
Zg = get_Zg(k, labels, groups)


In [250]:
Zg

array([[[0.        , 0.        , 0.        , 0.        , 0.03389831],
        [0.05084746, 0.        , 0.        , 0.        , 0.        ],
        [0.03389831, 0.        , 0.        , 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , 0.01694915, 0.        ],
        [0.        , 0.        , 0.01694915, 0.        , 0.        ],
        [0.        , 0.        , 0.01694915, 0.01694915, 0.        ]],

       [[0.        , 0.        , 0.        , 0.        , 0.03389831],
        [0.01694915, 0.        , 0.        , 0.        , 0.        ],
        [0.03389831, 0.        , 0.        , 0.        , 0.01694915],
        ...,
        [0.        , 0.        , 0.01694915, 0.01694915, 0.03389831],
        [0.        , 0.01694915, 0.        , 0.01694915, 0.        ],
        [0.        , 0.        , 0.        , 0.03389831, 0.01694915]],

       [[0.        , 0.        , 0.        , 0.        , 0.03389831],
        [0.03389831, 0.        , 0.        , 0.        , 0. 

In [251]:
Z = np.zeros((m,n,k))

In [252]:
for i in range(len(data)):
    Z[data.iloc[i, 1]-1, data.iloc[i, 0]-1, data.iloc[i, 2]-1] = 1


In [253]:
Zg = np.zeros((3,n,k))
for g in range(3):
    Zg[g,:,:] = np.mean(Z[groups==g,:,:],axis=0)

In [254]:
np.all(Zg == get_Zg(k, labels, groups))

True

In [24]:
M2s, M3s = get_M(Zg)


In [25]:
a,b,c = 1,2,0
Za = (Zg[c, :, :].T@Zg[b, :, :]/n)@np.linalg.inv(Zg[a, :, :].T @
                                                 Zg[b, :, :]/n)@Zg[a, :, :].T
Zb = (Zg[c, :, :].T@Zg[a, :, :]/n)@np.linalg.inv(Zg[b, :, :].T @
                                                 Zg[a, :, :]/n)@Zg[b, :, :].T

In [26]:
M3 = np.einsum("ji,ki,li->jkl", Za, Zb, Zg[c, :, :].T)/n
M2 = Za@Zb.T/n


In [27]:
u, s, vh = np.linalg.svd(M2, full_matrices=False)


In [28]:
get_whiten(M2, sym=True)


array([[  -9.19524314,   25.8762038 ,   59.94033902, -129.54118602,
          15.06207194],
       [ -13.53678217,   41.70697782,   56.48399218,   91.12466057,
        -107.97827229],
       [ -16.47236907,   42.96515193,  -22.52902475,   29.53756633,
         206.2432776 ],
       [ -20.49160929,   31.83289851,  -74.2551731 ,  -42.95577196,
        -130.40987372],
       [ -46.57817747,  -46.42863966,   12.38642101,    7.54231364,
          12.84231721]])

In [29]:
Q = u@np.diag(s**-0.5)

In [30]:
Q.shape

(5, 5)

In [31]:
M3_whiten = whiten_tensor(M3, Q)

In [287]:
M3_new = np.zeros((k,k,k))
for a in range(k):
    for b in range(k):
        for c in range(k):
            for d in range(k):
                for e in range(k):
                    for f in range(k):
                        M3_new[a,b,c]+=M3[d,e,f]*Q[d,a]*Q[e,b]*Q[f,c]

In [336]:
values, vectors = robust_tensor_power(M3_whiten, 20, 100)


In [292]:
np.allclose(M3_new,M3_whiten)

True

In [268]:
M3_2 = np.zeros((k,k,k))

In [269]:
for i1 in range(k):
    for i2 in range(k):
        for i3 in range(k):
            M3_2[i1, i2, i3] = np.sum(Za[i1,:]*Zb[i2,:]*Zg[c,:,i3])/n


In [275]:
M3_2


array([[[1.06549972e-06, 4.00909567e-07, 1.86179973e-07, 6.24276453e-08,
         1.03600769e-07],
        [2.54174415e-07, 4.46648428e-07, 1.40646002e-07, 7.01520550e-08,
         1.77188614e-07],
        [2.36062713e-07, 1.11559407e-07, 1.74379658e-07, 1.00333511e-07,
         1.17523448e-07],
        [5.52842310e-08, 6.02824313e-08, 1.31176047e-07, 2.14339296e-08,
         7.79550528e-08],
        [5.39319191e-08, 1.07918096e-07, 1.30878074e-07, 1.41321823e-07,
         2.78167546e-07]],

       [[3.37570139e-07, 3.50205452e-07, 1.47750442e-07, 1.00946691e-07,
         1.26257788e-07],
        [3.57496434e-07, 1.13783132e-06, 5.66940033e-07, 2.62104108e-07,
         1.53713051e-07],
        [1.10026953e-07, 4.33278892e-07, 4.48427205e-07, 2.72991611e-07,
         1.78693951e-07],
        [8.98954393e-08, 2.25697396e-07, 1.88496372e-07, 1.36317728e-07,
         1.52921146e-07],
        [1.52980207e-07, 2.17240718e-07, 1.19820034e-07, 7.11922876e-08,
         3.12528959e-07]],

      

In [276]:
M3

array([[[1.06549972e-06, 4.00909567e-07, 1.86179973e-07, 6.24276453e-08,
         1.03600769e-07],
        [2.54174415e-07, 4.46648428e-07, 1.40646002e-07, 7.01520550e-08,
         1.77188614e-07],
        [2.36062713e-07, 1.11559407e-07, 1.74379658e-07, 1.00333511e-07,
         1.17523448e-07],
        [5.52842310e-08, 6.02824313e-08, 1.31176047e-07, 2.14339296e-08,
         7.79550528e-08],
        [5.39319191e-08, 1.07918096e-07, 1.30878074e-07, 1.41321823e-07,
         2.78167546e-07]],

       [[3.37570139e-07, 3.50205452e-07, 1.47750442e-07, 1.00946691e-07,
         1.26257788e-07],
        [3.57496434e-07, 1.13783132e-06, 5.66940033e-07, 2.62104108e-07,
         1.53713051e-07],
        [1.10026953e-07, 4.33278892e-07, 4.48427205e-07, 2.72991611e-07,
         1.78693951e-07],
        [8.98954393e-08, 2.25697396e-07, 1.88496372e-07, 1.36317728e-07,
         1.52921146e-07],
        [1.52980207e-07, 2.17240718e-07, 1.19820034e-07, 7.11922876e-08,
         3.12528959e-07]],

      

In [39]:
def robust_tensor_power(T, L=20, N=100, sym=True):
    k, _, _ = T.shape
    if sym:
        T = sym_tensor(T)
    values = np.zeros(k)
    vectors = np.zeros((k, k))
    for h in range(k):
        theta_tau = []
        for tau in range(L):
            theta = np.random.randn(k)
            theta /= np.linalg.norm(theta)
            for t in range(N):
                theta = np.einsum("aef,e,f->a", T, theta, theta)
                theta /= np.linalg.norm(theta)
            theta_tau.append(theta)

        lam_best = float("-Inf")
        theta_best = np.array([])
        for theta in theta_tau:
            lam = np.einsum("efg,e,f,g->", T, theta, theta, theta)
            if lam > lam_best:
                lam_best = lam
                theta_best = theta

        theta = theta_best
        for t in range(N):
            theta = np.einsum("aef,e,f->a", T, theta, theta)
            theta /= np.linalg.norm(theta)
        lam = np.einsum("efg,e,f,g->", T, theta, theta, theta)

        values[h] = lam
        vectors[:, h] = theta
        T = T-lam*np.einsum("a,b,c->abc", theta, theta, theta)
    return values, vectors

In [133]:
lam, vec = robust_tensor_power(M3_whiten)

In [132]:
T = sym_tensor(M3_whiten)


In [121]:
lam, vec = robust_tensor_power(T, L = 20,N=100, sym=False)


In [122]:
temp = np.zeros((k, k, k))
for i in range(k):
    temp += lam[i]*np.einsum("a,b,c->abc", vec[:, i], vec[:, i], vec[:, i])

In [123]:
np.mean(np.abs(temp-T))

0.03568052611212279

In [134]:
mu = np.linalg.inv(Q.T)@vec@np.diag(lam)


In [165]:
def get_confusion_matrix(k, labels, groups=None, sym=True,cutoff=1e-7, L=20, N=100, seed=0):
    m, n = labels.shape
    if groups is None:
        np.random.seed(seed)
        groups = np.random.randint(3, size=m)
    Zg = get_Zg(k, labels, groups)
    M2s, M3s = get_M(Zg)
    Cc = np.zeros((3, k, k))
    W = np.zeros((3, k, k))
    for g, (M2, M3) in enumerate(zip(M2s, M3s)):
        Q = get_whiten(M2, sym)
        M3_whiten = whiten_tensor(M3, Q)
        values, vectors = robust_tensor_power(M3_whiten, L, N, sym)
        w = values**-2
        mu = np.linalg.inv(Q.T)@vectors@np.diag(values)
        best = np.argmax(mu, axis=0)
        for h in range(k):
            Cc[g, :, best[h]] = mu[:, h]
            W[g, best[h], best[h]] = w[h]
    W = np.mean(W, axis=0)
    C = np.zeros((m, k, k))
    for i in range(m):
        Ca = (np.sum(Cc, axis=0)-Cc[groups[i], :, :])/2
        Za = (np.sum(Zg, axis=0)-Zg[groups[i], :, :])/2
        E = np.zeros((k, k))
        for j in range(n):
            if labels[i,j]!=-1:
                E[labels[i, j], :] += Za[j, :]
        E /= n
        Ci = E@np.linalg.inv(W@Ca.T)
        if cutoff:
            Ci[Ci < cutoff] = cutoff
        colsums = np.sum(Ci, axis=0)
        Ci /= colsums[np.newaxis, :]
        C[i, :, :] = Ci
    return C


In [166]:
C_estimated = get_confusion_matrix(k, labels, groups=groups, sym=False)


In [136]:
np.argmax(mu,axis=1)

array([0, 2, 1, 4, 3], dtype=int64)

In [138]:
mu0 = lam[0]*np.linalg.inv(Q.T)@vec[:,0]


In [161]:
E = -np.eye(5)

In [163]:
E[E<0]=0

In [164]:
E

array([[ 0., -0., -0., -0., -0.],
       [-0.,  0., -0., -0., -0.],
       [-0., -0.,  0., -0., -0.],
       [-0., -0., -0.,  0., -0.],
       [-0., -0., -0., -0.,  0.]])

In [139]:
np.argmax(mu0)

0

In [82]:
robust_tensor_power(temp,sym=False)


(array([2.47221395, 1.9597989 , 1.68658776, 1.62258932, 1.5157169 ]),
 array([[-0.20816801, -0.45904245, -0.28248499, -0.78507705, -0.44330809],
        [ 0.34572185,  0.49387216,  0.52550452, -0.60544145,  0.20886257],
        [ 0.44722585, -0.25200777,  0.59235523,  0.1237719 , -0.73952102],
        [-0.78471769,  0.16964156,  0.39568989,  0.03355718, -0.24202772],
        [ 0.14611525,  0.67311525, -0.36958358, -0.02547738, -0.39291933]]))

In [83]:
robust_tensor_power(M3_whiten,sym=False)


(array([2.46460466, 1.95134792, 1.69621225, 1.61947364, 1.52595186]),
 array([[-0.19890427, -0.44754347, -0.27192651, -0.7790689 , -0.44857849],
        [ 0.33327075,  0.49026603,  0.52012714, -0.61032911,  0.20734864],
        [ 0.44301318, -0.23003884,  0.60929923,  0.1369673 , -0.7439836 ],
        [-0.79524795,  0.1743316 ,  0.39476733,  0.03790537, -0.24151606],
        [ 0.14383224,  0.68995267, -0.35838099, -0.01879279, -0.37939724]]))